# 💠 FinSight Protocol : Détection de Fraude Financière Avancée
**Auteur :** Antigravity (Google Deepmind Team)

---

## 🎯 Objectif
Développer un modèle de Machine Learning robuste capable de détecter des anomalies comptables et des tentatives de fraude dans un jeu de données financier déséquilibré.

**Méthodologie :**
1.  **Simulation Forensique :** Génération de données réalistes intégrant des indicateurs financiers (Liquidité, Levier, Marge) et comportementaux (Loi de Benford).
2.  **Traitement du Déséquilibre :** Utilisation de **SMOTE** (Synthetic Minority Over-sampling Technique) pour contrer la rareté des fraudes.
3.  **Modélisation :** Entraînement d'un classifieur **Random Forest**.
4.  **Évaluation :** Comparaison des performances avec et sans SMOTE.

In [ ]:
# 1. INSTALLATION ET IMPORT DES LIBRAIRIES
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA

# Configuration esthétique
sns.set_theme(style="darkgrid")
plt.rcParams['figure.figsize'] = (12, 6)

## 🛠️ Phase 1 : Génération de Données Forensiques (Simulation)
Nous simulons un dataset de 5000 entreprises avec 5% de fraude.

In [ ]:
def generate_financial_data(n_samples=5000, fraud_ratio=0.05):
    np.random.seed(42)
    n_frauds = int(n_samples * fraud_ratio)
    n_legit = n_samples - n_frauds
    
    # --- Profil des Entreprises Saines ---
    legit_data = pd.DataFrame({
        'current_ratio': np.random.normal(1.5, 0.3, n_legit),
        'debt_to_equity': np.random.normal(0.5, 0.1, n_legit),
        'net_margin': np.random.normal(0.10, 0.02, n_legit),
        'benford_deviation': np.random.exponential(0.05, n_legit),
        'text_complexity': np.random.normal(10, 2, n_legit),
        'is_fraud': 0
    })
    
    # --- Profil des Fraudeurs (Anomalies) ---
    fraud_data = pd.DataFrame({
        'current_ratio': np.random.normal(0.8, 0.4, n_frauds),
        'debt_to_equity': np.random.normal(1.2, 0.5, n_frauds),
        'net_margin': np.random.normal(0.15, 0.05, n_frauds),
        'benford_deviation': np.random.normal(0.3, 0.1, n_frauds),
        'text_complexity': np.random.normal(16, 3, n_frauds),
        'is_fraud': 1
    })
    
    df = pd.concat([legit_data, fraud_data], ignore_index=True)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    return df

df = generate_financial_data()
print(f"Dataset Shape: {df.shape}")
print(f"Distribution des classes :\n{df['is_fraud'].value_counts(normalize=True)}")

## ⚖️ Phase 2 : Le Problème du Déséquilibre & SMOTE

In [ ]:
X = df.drop(columns=['is_fraud'])
y = df['is_fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print(f"Entraînement Original - Fraudes : {sum(y_train == 1)} / Total : {len(y_train)}")

# Application SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print(f"Entraînement Après SMOTE - Fraudes : {sum(y_train_smote == 1)} / Total : {len(y_train_smote)}")

In [ ]:
# Visualisation de l'effet SMOTE via PCA
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_smote_pca = pca.transform(X_train_smote)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].scatter(X_train_pca[y_train==0, 0], X_train_pca[y_train==0, 1], label='Légitime', alpha=0.3, color='blue')
axes[0].scatter(X_train_pca[y_train==1, 0], X_train_pca[y_train==1, 1], label='Fraude', alpha=0.6, color='red')
axes[0].set_title("Avant SMOTE")
axes[0].legend()

axes[1].scatter(X_smote_pca[y_train_smote==0, 0], X_smote_pca[y_train_smote==0, 1], label='Légitime', alpha=0.3, color='blue')
axes[1].scatter(X_smote_pca[y_train_smote==1, 0], X_smote_pca[y_train_smote==1, 1], label='Fraude (Augmentée)', alpha=0.6, color='red')
axes[1].set_title("Après SMOTE")
axes[1].legend()

plt.show()

## 🧠 Phase 3 : Entraînement et Comparaison (Random Forest)

In [ ]:
# Modèle 1 : Standard
rf_standard = RandomForestClassifier(n_estimators=100, random_state=42)
rf_standard.fit(X_train, y_train)
y_pred_std = rf_standard.predict(X_test)

# Modèle 2 : SMOTE
rf_smote = RandomForestClassifier(n_estimators=100, random_state=42)
rf_smote.fit(X_train_smote, y_train_smote)
y_pred_smote = rf_smote.predict(X_test)

def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(title)
    plt.xlabel('Prédiction')
    plt.ylabel('Réalité')
    plt.xticks([0.5, 1.5], ['Légitime', 'Fraude'])
    plt.yticks([0.5, 1.5], ['Légitime', 'Fraude'])
    plt.show()

print("--- SANS SMOTE ---")
print(classification_report(y_test, y_pred_std))
plot_confusion_matrix(y_test, y_pred_std, "Sans SMOTE")

print("--- AVEC SMOTE ---")
print(classification_report(y_test, y_pred_smote))
plot_confusion_matrix(y_test, y_pred_smote, "Avec SMOTE")

## ✅ Conclusion
L'utilisation de **SMOTE** a permis de rééquilibrer la sensibilité du modèle, augmentant significativement la détection des fraudes.